# PersonBIO

In [180]:
seed = 1
base_path = "../workdir/fc_new/bio/mistralai/Mistral-7B-Instruct-v0.1/['rediska0123/person-bio']/"
time = "2024-10-05/15-35-38"
data = {}


for fold in range(5):
    path1 = f"{base_path}/{time}/ue_manager_seed{seed}_fold{fold}"

    data1 = torch.load(path1)
    
    for k in data1.keys():
        if isinstance(data1[k], dict):
            if k not in data.keys():
                data[k] = {}
            for k1 in data1[k].keys():
                if isinstance(data1[k][k1], list):
                    if k1 not in data[k].keys():
                        data[k][k1] = data1[k][k1]
                    else:
                        data[k][k1] += data1[k][k1]
                else:
                    if k1 not in data[k].keys():
                        data[k][k1] = data1[k][k1]
                    else:
                        data[k][k1] += data1[k][k1]
                        
        elif isinstance(data1[k], list):
            data[k] = data1[k] + data2[k]
        else:
            print(k)

/tmp/ipykernel_129/2282314469.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data1 = torch.load(path1)


In [181]:
np.nanmean(data['gen_metrics'][('claim', 'OpenAIFactCheck')])

0.18336776859504134

In [182]:
from lm_polygraph.ue_metrics import *

rocauc = ROCAUC()
prauc = PRAUC()

def preproc_method_name(x):
    return x

def get_table_upd(man, metric_names=['PR-AUC', "ROC-AUC"], dataset="FactChecking", ue_metric="prr_normalized", 
                  update_alignscore=False, aggregated=False, return_score=False, level="claim"):
    df = {}
    methods = np.unique([x[1] for x in man['metrics'].keys() if "2side" not in x[1]])
    metric = np.array(man['gen_metrics'][('claim', 'OpenAIFactCheck')])  
    for method in methods:
        key = (level, method, "", ue_metric)
        level = "claim"
        estimator_values = np.array(man['estimations'][(level, key[1])]).flatten()         
        rec_metrics_val = np.array(metric)

        ue = np.array(estimator_values)
        ue = np.nan_to_num(ue, 0)
        final_score = prauc(ue, rec_metrics_val) 
        scores = df.get(method, [])
        scores.append(final_score)
        df[method] = scores

    for method in methods:
        key = (level, method, "", ue_metric)
        level = "claim"
        estimator_values = np.array(man['estimations'][(level, key[1])]).flatten()         
        rec_metrics_val = np.array(metric)

        ue = np.array(estimator_values)
        ue = np.nan_to_num(ue, 0)
        final_score = rocauc(ue, rec_metrics_val) 
        scores = df.get(method, [])
        scores.append(final_score)
        df[method] = scores

    df0 = pd.DataFrame(df, index=pd.MultiIndex.from_tuples([(dataset, m) for m in metric_names])).T
    method = [preproc_method_name(x) for x in df0.index]
    df0.loc[:, (dataset, 'Method')] = method
    df0 = df0[df0.columns[[2,1,0]]]
    return df0.reset_index(drop=True)

In [183]:
df_fc = get_table_upd(data)
df_fc.style.background_gradient(axis=0)

In [184]:
def preproc_method_name(x):
    label = x
    if "HUQ" in x:
        label = "HUQ-STMD"
    elif "Hybrid" in x:
        label = "STMD+CCP"
    elif "LinReg" in x:
        label = "STMD"
    elif "Fact" in x:
        label = "Factoscope"
    elif "SAPLMA" in x:
        label = "SAPLMA"
    elif "EigenScore" in x:
        label = "EigenScore"
    elif "CCP" in x:
        label = "CCP"
    elif "PTrueClaim" in x:
        label = "P(True)"
    elif "MaximumClaimProbability" in x:
        label = "Maximum Claim Probability"
        
    if "relative" in x.lower():
        label = label.replace("TMD", "TRMD")
    return label
    
baselines = ['MaximumClaimProbability', 'PTrueClaim', 'CCP_claim_fact_pref']
cols = df_fc[df_fc.columns[0]]
n_layers = len([x for x in cols if "SAPLMAClaim_decoder" in x])
mid_layer = int(n_layers // 2)
density_baselines = [f"SAPLMAClaim_decoder_{mid_layer} ()"]
proposed_methods = [col for col in cols if "LinReg" in col and "CCP_claim_fact_pref_decoder" in col and "remove_corr_3" in col] + [col for col in cols if "LinReg" in col and "Hybrid" not in col and "HUQ" not in col and "remove_corr_3" in col]

final_methods = baselines + density_baselines + proposed_methods
df_fc = df_fc[df_fc[df_fc.columns[0]].isin(final_methods)]
# df0 = df0.loc[final_methods]
methods = [preproc_method_name(x) for x in df_fc[df_fc.columns[0]]]
df_fc[df_fc.columns[0]] = methods
df_fc = df_fc.reset_index(drop=True)

/tmp/ipykernel_129/2319378298.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fc[df_fc.columns[0]] = methods


In [185]:
df_fc

FactChecking                    
                      Method   ROC-AUC    PR-AUC
0                        CCP  0.715510  0.387543
1                  HUQ-STRMD  0.750055  0.409947
2                   HUQ-STMD  0.735739  0.398407
3                  STRMD+CCP  0.739221  0.413827
4                   STMD+CCP  0.734669  0.413019
5                      STRMD  0.646794  0.274952
6                       STMD  0.638972  0.267418
7  Maximum Claim Probability  0.620230  0.270953
8                    P(True)  0.638334  0.275904
9                     SAPLMA  0.489454  0.165920

In [186]:
order = [7,8,0,9,5,1,3]
df_fc = df_fc.iloc[order].reset_index(drop=True)

In [187]:
df_fc.style.background_gradient(axis=0)

In [29]:
import matplotlib 
from matplotlib import colors
import matplotlib.pyplot as plt 

cmap = matplotlib.cm.get_cmap('coolwarm')#'Greens')
my_cmap = cmap(np.arange(cmap.N))
my_cmap[:,-1] = 0.5
my_cmap = colors.ListedColormap(my_cmap)

def b_g(s, cmap, low=0, high=0):
    values = s.apply(lambda x: float(x.split("±")[0]) if len(x.split("±"))>1 else x)
    if isinstance(values.max(), str):
        return ['' for c in values]
    rng = values.max() - values.min()
    norm = colors.Normalize(values.min() - (rng * low), values.max() + (rng * high))
    normed = norm(values.values)
    back_colors = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    text_colors = ["white" if x>0.3 else "black" for x in normed]
    return [f'color: {text_color}; background-color: {color}' for text_color, color in zip(text_colors, back_colors)]

/tmp/ipykernel_129/1380247612.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('coolwarm')#'Greens')


In [30]:
def rgba2rgb(rgba, background=(1,1,1)):
    ch = rgba.shape[0]
    if ch == 3:
        return rgba

    assert ch == 4, 'RGBA image has 4 channels.'

    r, g, b, a = rgba[0], rgba[1], rgba[2], rgba[3]
    a = np.asarray(a, dtype='float32')
    R, G, B = background

    r_new = r * a + (1.0 - a) * R
    g_new = g * a + (1.0 - a) * G
    b_new = b * a + (1.0 - a) * B

    return [r_new, g_new, b_new]


def to_color(text, vals, remove_zeros, bold=False, underline=False):
    vals = rgba2rgb(np.array(vals))
    if remove_zeros:
        number = f'{text:.3f}'
        if number[0] == "-":
            number = number[:1] + number[2:]
        else:
            number = number[1:]
        if bold:
            return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+ "\\textbf{"+number+"}"
        if underline:
            return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+ "\\underline{"+number+"}"
        return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+ number
    if bold:
        return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+ "\\textbf{"+f'{text:.2f}'+"}"
    if underline:
        return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+ "\\underline{"+f'{text:.2f}'+"}"
    return '\\cellcolor[rgb]{'+f'{vals[0]},'+f'{vals[1]},'+f'{vals[2]}'+'} '+f'{text:.2f}'

def bold_best(df, columns, remove_zeros):
    for col in columns:
        values_init_raw = [float(x) if x!='-' else np.nan for x in df[col]]
        values_init = np.array([float(x) for x in df[col] if x!='-'])
        if values_init.min() != values_init.max():
            values_init_raw = np.array([(x - values_init.min()) / (values_init.max() - values_init.min()) if not np.isnan(x) else x for x in values_init_raw])
            
        def get_new_x(x):
            if isinstance(x, str):
                return x
            return '-'

        def preproc_x(x):
            # if remove_zeros:
            #     return int(x*1000)
            return x

        remove_zeros = True
        if "rank" in col[-1].lower():
            remove_zeros = False
        if "impr" in col[-1].lower():
            remove_zeros = False
            
        if "rank" not in col[-1].lower():
            max_val = df[col].replace('-', 0).nlargest(2)
            values = [to_color(preproc_x(raw), my_cmap(float(x)), remove_zeros, bold=(raw==max_val.iloc[0]), underline=(raw==max_val.iloc[1])) if (isinstance(x, float) and (not np.isnan(x))) else get_new_x(x) for raw, x in zip(df[col], values_init_raw)]
        else:
            max_val = df[col].replace('-', 0).nsmallest(2)
            values = [to_color(preproc_x(raw), my_cmap(1-float(x)), remove_zeros, bold=(raw==max_val.iloc[0]), underline=(raw==max_val.iloc[1])) if (isinstance(x, float) and (not np.isnan(x))) else get_new_x(x) for raw, x in zip(df[col], values_init_raw)]
        df[col] = values
    return df

def prepare_latex(df1, s=2, remove_zeros=False):
    start_tex = '\\begin{table*}[!ht] \\resizebox{\\textwidth}{!}{'
    end_tex = "}\\caption{\\label{tab:mistral_results} PRR$\\uparrow$ for Mistral 7b v0.2 model for various tasks for the considered sequence-level methods. Darker color indicates better results.}\end{table*}"
    df1 = bold_best(df1, df1.columns[s:], remove_zeros)
    latex_table = df1.to_latex(bold_rows=False, index=False).replace('±', '$\pm$')
        
    latex_table = latex_table.replace('\\textbackslash ', '\\')
    latex_table = latex_table.replace('{lllllllllllllllllllll}', '{ll||cc|cc|cc|cc|cc|cc|cc|cc|cc|c}')
    latex_table = latex_table.replace('{lrllllllllllllllllllll}', '{l||cc|cc|cc|cc|cc|cc|cc|cc|cc|c|c|c}')
    latex_table = latex_table.replace('{llllllllllllllllllllll}', '{l||cc|cc|cc|cc|cc|cc|cc|cc|cc|c|c|c}')
    latex_table = latex_table.replace('{llllllllllllllllllll}', '{l||cc|cc|cc|cc|cc|cc|cc|cc|cc|c}')
    latex_table = latex_table.replace('{lrllllllllllllllllll}', '{l||cc|cc|cc|cc|cc|cc|cc|cc|cc|c}')
    latex_table = latex_table.replace('{lllllllllllllll}', '{ll||cc|cc|cc|cc|cc|cc|c}')
    latex_table = latex_table.replace('{lllllllll}', '{ll||cc|cc|cc|c}')
    latex_table = latex_table.replace('{llllllll}', '{l||cc|cc|cc|c}')
    latex_table = latex_table.replace('{lll}', '{l||cc}')
    

    latex_table = latex_table.replace('mean', '$\\frac{1}{K}\\sum_{k=1}^K p_k$')
    latex_table = latex_table.replace('sum(log(p_i))', '$\\sum_{k=1}^K \\log p_k$')

    latex_table = latex_table.replace('\\{', '{')
    latex_table = latex_table.replace('\\}', '}')
    str_list = latex_table.split('\n')
    str_list.pop(3)
    latex_table = '\n'.join(str_list)
    return start_tex+latex_table+end_tex

In [31]:
import copy

with pd.option_context("max_colwidth", 1000):
    res_str = prepare_latex(copy.deepcopy(df_fc).round(3),1,True).split('\n')
    res_str[2] = """\\multirow{2}{*}{\\textbf{UQ Method}} & \multicolumn{2}{c|}{\\textbf{Mistral 7b}} \\\\ \\cline{2-3}
    & \\textbf{ROC-AUC} & \\textbf{PR-AUC}  \\\\"""
    print('\n'.join(res_str))

\begin{table*}[!ht] \resizebox{\textwidth}{!}{\begin{tabular}{l||cc}
\toprule
\multirow{2}{*}{\textbf{UQ Method}} & \multicolumn{2}{c|}{\textbf{Mistral 7b}} \\ \cline{2-3}
    & \textbf{ROC-AUC} & \textbf{PR-AUC}  \\
\midrule
Maximum Claim Probability & \cellcolor[rgb]{0.9337138175431372,0.9321882998862745,0.9313012310098039} .620 & \cellcolor[rgb]{0.8910245585529412,0.9214321063294117,0.9714899216352941} .271 \\
P(True) & \cellcolor[rgb]{0.9653342981666666,0.909438499827451,0.8795731953490196} .638 & \cellcolor[rgb]{0.9024823794117647,0.9258330802784314,0.9630825372156863} .276 \\
CCP & \cellcolor[rgb]{0.9367011412764705,0.6934798195294117,0.6531662319882353} .716 & \cellcolor[rgb]{0.9218513348607843,0.6650340647294117,0.635032405109804} .388 \\
SAPLMA & \cellcolor[rgb]{0.61490285,0.649358983,0.8768415764999999} .489 & \cellcolor[rgb]{0.61490285,0.649358983,0.8768415764999999} .166 \\
STRMD & \cellcolor[rgb]{0.974575254145098,0.8953926345333334,0.8554377971509803} .647 & \cellcolor[rg